In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import time

In [2]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth

CLIENT_ID = 'ClientID'
CLIENT_SECRET = 'Client Secret'
REDIRECT_URI = ''

scope = ["user-library-read", "playlist-read-private"]

sp = Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                       client_secret=CLIENT_SECRET,
                                       redirect_uri=REDIRECT_URI,
                                       scope=scope),
             requests_timeout=30)

# Kullanıcı bilgilerini yazdır (kontrol amaçlı)
user = sp.current_user()
print(user["display_name"], "-", user["id"])


Oğulcan - 4j17p063fi94racfiau0ksd6c


In [4]:
token_info = sp.auth_manager.get_cached_token()
if token_info:
    access_token = token_info['access_token']
    print("Access Token:", access_token)
else:
    print("Token bulunamadı.")


Access Token: BQADriKVJZN5KgcITY05pdrGptyi-FbMujHD3UHE2cwuNcpU9helbXyfF2r6MYwXQJTw0LkExgcyKlnxOnXXhcLtHVSmeXw6OtKBA-c6T4CDtYzgoEKU8Olm_vMsaqKkp51WQxxDD4AThFzkwgJTGwVQOfW5KXsAv8Ad8l4VFk3FLcdSIIUy1TdY2Aj72oR75ItNvXMkYZMQpJGagDXI_cBsFwklTi0r8MiZit5ypTGKmx8yyvz0-iHMuquV63MQJg


In [ ]:
# Playlist ID'yi buraya gir
playlist_id = "6W57V9nC35ul5GRg2J5y29"

# Playlist’ten tüm şarkıları al
def get_all_tracks(sp, playlist_id):
    tracks = []
    results = sp.playlist_tracks(playlist_id, limit=100, offset=0)
    tracks.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Her şarkının audio feature'larını al
def get_audio_features(sp, track_ids):
    features = []
    for i in range(0, len(track_ids), 50):
        batch = track_ids[i:i+50]
        try:
            audio_features = sp.audio_features(batch)
            features.extend([f for f in audio_features if f is not None])
            time.sleep(0.2)
        except spotipy.exceptions.SpotifyException as e:
            print(f"API hatası: {e}")
            continue
    return features

# 1. Şarkıları al
tracks_data = get_all_tracks(sp, playlist_id)
track_ids = [item['track']['id'] for item in tracks_data if item['track']]

# 2. Audio feature'ları al
features_data = get_audio_features(sp, track_ids)

# 3. DataFrame'e dönüştür
df_features = pd.DataFrame(features_data)

# 4. Track isimlerini ekle
track_names = [item['track']['name'] for item in tracks_data if item['track']]
df_features['track_name'] = track_names

# Sonuç
df_features



HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4qKcDkK6siZ7Jp1Jb4m0aL,6T8cJz5lAqGer9GUHGyelE,0gV5B7zmJkS1aaH9APokrH,5S1IUPueD0xE0vj4zU3nSf,46gjYTEK7W8ZTABcglGP2f,3dgQqOiQ9fCKVhNOedd2lf,62vpWI1CHwFy7tMIcSStl8,2Vx8E3K5icPZR7OCklWBXX,7p7pzSVr2zovan3m1m5hFm,0bAkKNCQfWkexHFn7fIKns,7rC5Pl8rQSX4myONQHYPBK,5R691ipUYRDYW6ehapjoj6,4XoP1AkbOurU9CeZ2rMEz2,2xLMifQCjDGFmkHkpNLD9h,6OaVWaGfhXn70ZJfdVcabn,1xYpVRspgaLxdJLOTQkTjg,6Ius4TC0L3cN74HT7ENE6e,4KYS9GK68yeuUJ8vJfPxrq,6gBFPUFcJLzWGx4lenP6h2,5e574bhjycX1eH2l4Auage,04MLEeAMuV9IlHEsD8vF6A,1AqemxWsT0iGg5Lvkm16Bt,2IRZnDFmlqMuOrYOLnZZyc,6MWtB6iiXyIwun0YzU6DFP,1rlIrbWboTRGeKfHhgbJRZ,7floNISpH8VF4z4459Qo18,5tFep7dXGd7vEJ668wTPux,2JvzF1RMd7lE3KmFlsyZD8,1knDG6hRiyjs3d7tG4GJ2Z,79XrkTOfV1AqySNjVlygpW,7DcvwMAiqKJQD1rrdfxSDx,4k2l5T9ulYdnlQzlmzSqUI,5ry2OE6R2zPQFDO85XkgRb,6UjfByV1lDLW0SOVQA4NAi,5cXg9IQS34FzLVdHhp7hu7,6MF4tRr5lU8qok8IKaFOBE,7ycWLEP1GsNjVvcjawXz3z,2jTujnt0y344ai1rNOywgr,2hyF1YWdX8yxExvRelT9nB,7KZ5MMVgBVox9ycroB2UrI,2cYqizR4lgv

API hatası: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=4qKcDkK6siZ7Jp1Jb4m0aL,6T8cJz5lAqGer9GUHGyelE,0gV5B7zmJkS1aaH9APokrH,5S1IUPueD0xE0vj4zU3nSf,46gjYTEK7W8ZTABcglGP2f,3dgQqOiQ9fCKVhNOedd2lf,62vpWI1CHwFy7tMIcSStl8,2Vx8E3K5icPZR7OCklWBXX,7p7pzSVr2zovan3m1m5hFm,0bAkKNCQfWkexHFn7fIKns,7rC5Pl8rQSX4myONQHYPBK,5R691ipUYRDYW6ehapjoj6,4XoP1AkbOurU9CeZ2rMEz2,2xLMifQCjDGFmkHkpNLD9h,6OaVWaGfhXn70ZJfdVcabn,1xYpVRspgaLxdJLOTQkTjg,6Ius4TC0L3cN74HT7ENE6e,4KYS9GK68yeuUJ8vJfPxrq,6gBFPUFcJLzWGx4lenP6h2,5e574bhjycX1eH2l4Auage,04MLEeAMuV9IlHEsD8vF6A,1AqemxWsT0iGg5Lvkm16Bt,2IRZnDFmlqMuOrYOLnZZyc,6MWtB6iiXyIwun0YzU6DFP,1rlIrbWboTRGeKfHhgbJRZ,7floNISpH8VF4z4459Qo18,5tFep7dXGd7vEJ668wTPux,2JvzF1RMd7lE3KmFlsyZD8,1knDG6hRiyjs3d7tG4GJ2Z,79XrkTOfV1AqySNjVlygpW,7DcvwMAiqKJQD1rrdfxSDx,4k2l5T9ulYdnlQzlmzSqUI,5ry2OE6R2zPQFDO85XkgRb,6UjfByV1lDLW0SOVQA4NAi,5cXg9IQS34FzLVdHhp7hu7,6MF4tRr5lU8qok8IKaFOBE,7ycWLEP1GsNjVvcjawXz3z,2jTujnt0y344ai1rNOywgr,2hyF1YWdX8yxExvRelT9nB,7KZ5MMVgBVox9yc

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=3yaYgjEFkRw3PVjW9mV1TO,2tUL6dZf1mywCj5WvCPZw6,40iJIUlhi6renaREYGeIDS,7F9cT6hIRhnFCYP6GKS0tf,3i952xMg7getPjzoOukIWh,1XKTvkeaZy5gueNNZYL7rE,6c5wQFfJApRMooKE7UQnlH,2oFdrFpXDgPcszmfWaKboI,4R67rQNSbbsR4TdUVOIdez,08F16baYbciTva9P4BvpiI,3szf2z1Cy1QMrtHrbn8rz9,1dUHF4RyMmMTveJ0Rby6Xm,6jy9yJfgCsMHdu2Oz4BGKX,6WSBZroxQaO3FYEpT2ACFH,4D1eVq5AUILwjg3tAe7o6M,0UTyLmkQgPOEIz95OVAJTR,30KctD1WsHKTIYczXjip5a,6AshXllQhobwSXsdpgp41w,5W8jRrZ6tWrTrqnKRtIQBf,08LwMWf5Tcfsd752EPzFV0,7EulRO07ZgchxqKUHb8Pvc,4fHRUyNTNVD0HRe4YPYA4m,4WEuEfFiczWxblHyuucSBL,4nuAslShoN77tq12fzwjUq,6myUpr3GDR80Dg3zqNTmmG,3f2k8op0nWDoZM4pXim6wG,2cnKST6T9qUo2i907lm8zX,4KDNRh9Oor80z3XIxdWlui,4n6NDfYake476trCjJRNl0,7Bpx2vsWfQFBACRz4h3IqH,3iVcZ5G6tvkXZkZKlMpIUs,3qX1dqmDdmMvvfMVfsx4Bh,6PzDwzYlw2gH6pYFQqNnFA,0JEefF2thwTsgWE2Wv0IsC,5pQZullzXsixizZmBcmXhy,2lD6AoA8qf2t4Dkf2TcmNK,6rTInqW3YECMkQsBEHw4sd,69WMdZSqdQwFgLVpbMsZ2J,4o7sl87Ock4SGA8b3gRR2l,6OrTKLtcF7EBayOV2QqkSK,0N3W5peJUQt

API hatası: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3yaYgjEFkRw3PVjW9mV1TO,2tUL6dZf1mywCj5WvCPZw6,40iJIUlhi6renaREYGeIDS,7F9cT6hIRhnFCYP6GKS0tf,3i952xMg7getPjzoOukIWh,1XKTvkeaZy5gueNNZYL7rE,6c5wQFfJApRMooKE7UQnlH,2oFdrFpXDgPcszmfWaKboI,4R67rQNSbbsR4TdUVOIdez,08F16baYbciTva9P4BvpiI,3szf2z1Cy1QMrtHrbn8rz9,1dUHF4RyMmMTveJ0Rby6Xm,6jy9yJfgCsMHdu2Oz4BGKX,6WSBZroxQaO3FYEpT2ACFH,4D1eVq5AUILwjg3tAe7o6M,0UTyLmkQgPOEIz95OVAJTR,30KctD1WsHKTIYczXjip5a,6AshXllQhobwSXsdpgp41w,5W8jRrZ6tWrTrqnKRtIQBf,08LwMWf5Tcfsd752EPzFV0,7EulRO07ZgchxqKUHb8Pvc,4fHRUyNTNVD0HRe4YPYA4m,4WEuEfFiczWxblHyuucSBL,4nuAslShoN77tq12fzwjUq,6myUpr3GDR80Dg3zqNTmmG,3f2k8op0nWDoZM4pXim6wG,2cnKST6T9qUo2i907lm8zX,4KDNRh9Oor80z3XIxdWlui,4n6NDfYake476trCjJRNl0,7Bpx2vsWfQFBACRz4h3IqH,3iVcZ5G6tvkXZkZKlMpIUs,3qX1dqmDdmMvvfMVfsx4Bh,6PzDwzYlw2gH6pYFQqNnFA,0JEefF2thwTsgWE2Wv0IsC,5pQZullzXsixizZmBcmXhy,2lD6AoA8qf2t4Dkf2TcmNK,6rTInqW3YECMkQsBEHw4sd,69WMdZSqdQwFgLVpbMsZ2J,4o7sl87Ock4SGA8b3gRR2l,6OrTKLtcF7EBayO

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=0LnS7aOdOdI1dNKZqdOLz4,1AufKRy3qVHdHmbwusee6A,6VPNLfB45fVbispXeNNHDB,46s57QULU02Voy0Kup6UEb,2r6OAV3WsYtXuXjvJ1lIDi,3oHkMCVJyOcjg5FhfLc2Rq,3DWjFSDEQ9IWz1mj449fB7,57BGVV6wcyhbn3hsjlqEZB,1bDbXMyjaUIooNwFE9wn0N,5unjCay0kUjuej5ebn4nS4,2KLwPaRDOB87XOYAT2fgxh,5H4mXWKcicuLKDn4Jy0sK7,3EMp20j5E42MxfFbsEsIvD,433P7tDcIAi6NLnf4Sh6tI,2ZL7WZcjuYKi1KUDtp4kCC,34tz0eDhGuFErIuW3q4mPX,45pUWUeEWGoSdH6UF162G8,5BXGLYOc7VY1YZsrLYDOxZ,12CNybio0Bxean3F3uYugU,1pRkGgpQGX6nr8PrO1NMIs,3GVkPk8mqxz0itaAriG1L7,2rFTRICumonxxZKxxfEu8R,4N9QsmvQJcGwJHAmDmBttU,7CC6UbCs4iGsePSzFxYxNn,1Is8hGpkGMiePASAxBluxM,0YImOCkIJ2PWhCXaURCZnY,792HwhrdO3ErRKL5yRe4Ge,609mekAwxC90JkQ4BDo9IK,7wTA0NKIm6T7nP2kaymU2a,4wNKxMbE3n1gRm4F6OaCP1,6gbiTbclnHlmSIPfmF2zEc,5ClRKRmW1p6RQcdwzY1kNJ,5Ee4IyhPaOVRMc15fUOeG9,4WuOWVnAqvEQxgSRrspBgt,7INo35yvWnVY5PcEUlg1d4,14qNHvX8h6HoynFuV0RxWS,5hVghJ4KaYES3BFUATCYn0,2FqqVHvC4eKrsk97vpMRid,2GAhgAjOhEmItWLfgisyOn,4KW1lqgSr8TKrvBII0Brf8,6tZ3b7ik1QD

...

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1lloYB6uHOTDmheiWsq4DL,6et5HwX6nTYddg7hDGAxug,3iRCr53b2ZOvoKMTNvBn5d,6bpFKtmybxjGaO7OSOQqOc,3rXlcLZk3MCaRPND5g9QiW,1Es7AUAhQvapIcoh3qMKDL,1KmmViIziBGJQa728cXkzh,1ITpqT1kDATCtMBwOK7sYn,33zbOXJSwW7uf4VXXWow2O,124IHGAzY9F3unizZ08iRc,7CyPwkp0oE8Ro9Dd5CUDjW,1wGnp7ejkCuXRsehXx0PNP,1tTKy8pW1dMaXJm7bYrSzA,3TN2HZSll8ueBFtt88VbfX,5emD8K39MmotlCLOx9qzCH,0NUqi0ps17YpLUC3kgsZq0,2u9S9JJ6hTZS3Vf22HOZKg,5Nz7hI3cCOHmMR4vSLJ1An,3FHO87BSOEVAsedSnCTnvz,0aB0v4027ukVziUGwVGYpG,4y1kHTpSNXM4VeCUtEFA1m,2Adxpr1j5lSihOCsmIfflJ,76gcXhY3Zv6wW0BTe9nHJo,0a4SMs889NwP8oWI7Vemle,0bkV1iQHSxBaksUqgEkcbc,5BwQjRasNcdRPuVWKcHto2,2cH8wHImerFokgmhjyhCTU,3IU6xbfNZosvTdRrGs8b2u,20MuVazoNMv6xjKPnRFOxG,1cZlBZwnwGPtYeRIeQcoFh,5SIvP6TdWc9DNvKbENjnYc,68qeaZhtMZ6abrJCYt6nQn,2n9fC0A4ptmWqYeMXEVaok,1pzN8bCzy017iK3vWzkk6Z,6iycYUk3oB0NPMdaDUrN1w,3at6WxcwxONcdbvcW02OOY,0uMsWm3xNlXkQ7q78Q0QKq,2YVkpynbH4lrZQZSJiaICM,2RWlRSi9TthgUka2yrLnrD,1RDBMVHirRQ62AvujJPCZA,53WWD2BBC6J

API hatası: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=1lloYB6uHOTDmheiWsq4DL,6et5HwX6nTYddg7hDGAxug,3iRCr53b2ZOvoKMTNvBn5d,6bpFKtmybxjGaO7OSOQqOc,3rXlcLZk3MCaRPND5g9QiW,1Es7AUAhQvapIcoh3qMKDL,1KmmViIziBGJQa728cXkzh,1ITpqT1kDATCtMBwOK7sYn,33zbOXJSwW7uf4VXXWow2O,124IHGAzY9F3unizZ08iRc,7CyPwkp0oE8Ro9Dd5CUDjW,1wGnp7ejkCuXRsehXx0PNP,1tTKy8pW1dMaXJm7bYrSzA,3TN2HZSll8ueBFtt88VbfX,5emD8K39MmotlCLOx9qzCH,0NUqi0ps17YpLUC3kgsZq0,2u9S9JJ6hTZS3Vf22HOZKg,5Nz7hI3cCOHmMR4vSLJ1An,3FHO87BSOEVAsedSnCTnvz,0aB0v4027ukVziUGwVGYpG,4y1kHTpSNXM4VeCUtEFA1m,2Adxpr1j5lSihOCsmIfflJ,76gcXhY3Zv6wW0BTe9nHJo,0a4SMs889NwP8oWI7Vemle,0bkV1iQHSxBaksUqgEkcbc,5BwQjRasNcdRPuVWKcHto2,2cH8wHImerFokgmhjyhCTU,3IU6xbfNZosvTdRrGs8b2u,20MuVazoNMv6xjKPnRFOxG,1cZlBZwnwGPtYeRIeQcoFh,5SIvP6TdWc9DNvKbENjnYc,68qeaZhtMZ6abrJCYt6nQn,2n9fC0A4ptmWqYeMXEVaok,1pzN8bCzy017iK3vWzkk6Z,6iycYUk3oB0NPMdaDUrN1w,3at6WxcwxONcdbvcW02OOY,0uMsWm3xNlXkQ7q78Q0QKq,2YVkpynbH4lrZQZSJiaICM,2RWlRSi9TthgUka2yrLnrD,1RDBMVHirRQ62Av

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4AmISPiQJMPLN4KootjCUu with Params: {} returned 403 due to None


API hatası: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=4AmISPiQJMPLN4KootjCUu:
 None, reason: None


,track_name
0,Look Alive (feat. Drake)
1,God's Plan
2,Wow Freestyle (feat. Kendrick Lamar)
3,Bigger Than You (feat. Drake & Quavo)
4,Jumpman


In [9]:
df_features

,track_name
0,Look Alive (feat. Drake)
1,God's Plan
2,Wow Freestyle (feat. Kendrick Lamar)
3,Bigger Than You (feat. Drake & Quavo)
4,Jumpman
...,...
346,FRIED
347,Sky
348,Rookie Of The Year
349,Livin' My Truth


In [10]:
# 1. Şarkı isimleriyle ID'leri alalım
track_ids = []
for track_name in df_features['track_name']:
    results = sp.search(q=track_name, limit=1)  # Her şarkıyı tek tek arıyoruz
    track_id = results['tracks']['items'][0]['id']  # Şarkı ID'sini alıyoruz
    track_ids.append(track_id)

# 2. Şarkı ID'lerini df_features'e ekleyelim
df_features['track_id'] = track_ids

# Kontrol etmek için ilk birkaç satırı görüntüleyelim
print(df_features.head())


                              track_name                track_id
0               Look Alive (feat. Drake)  6HJ34Zyw6bg8yGm28AxLXf
1                             God's Plan  5t9KtCfnu8wQui1G6IqqBR
2   Wow Freestyle (feat. Kendrick Lamar)  0gV5B7zmJkS1aaH9APokrH
3  Bigger Than You (feat. Drake & Quavo)  5S1IUPueD0xE0vj4zU3nSf
4                                Jumpman  27GmP9AWRs744SzKcpJsTZ


In [18]:
# Spotify API'den audio özelliklerini almak için şarkı ID'lerini gruplara ayıralım
def fetch_audio_features_in_chunks(track_ids, chunk_size=20):
    features = []
    for i in range(0, len(track_ids), chunk_size):
        chunk = track_ids[i:i + chunk_size]
        try:
            # API'yi her seferinde küçük bir grup ile sorgulama
            results = sp.audio_features(chunk)
            features.extend(results)
        except Exception as e:
            print(f"Error while fetching features for chunk {i // chunk_size + 1}: {e}")
    return features

# Şarkı ID'lerini 100'lü gruplara ayırarak audio özelliklerini çekme
track_ids = df_features['track_id'].tolist()  # track_id'lerin olduğu liste
audio_features = fetch_audio_features_in_chunks(track_ids)


Error while fetching features for chunk 1: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error while fetching features for chunk 2: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=2cYqizR4lgvp4Qu6IQ3qGN,0uFz9q2SvuKyWB6Ur1YCfI,1wHZx0LgzFHyeIZkUydNXq,6KfoDhO4XUWSbnyKjNp9c4,3eekarcy7kvN4yt5ZFzltW,6CfrYuD3YRDYdYvH9jNtXY,0utlOiJy2weVl9WTkcEWHy,5InOp6q2vvx0fShv3bzFLZ,4kdfjhj9xNkYU0R8xlDy8k,0zoGVO4bQXG8U6ChKwNgeg,3GVkPk8mqxz0itaAriG1L7,5lBzjMCr2QRoNlleZq6l7P,7uSGU4SrX7NyaqGTueSZf0,6REbwUNlppTfcnV4d4ZoZi,6gi6y1xwmVszDWkUqab1qw,43uQtNJUmlEdJtjbz6GdGL,5KpUzmHFirESqWjCrXnmWS,5yY9lUy8nbvjM1Uyo1Uqoc,12d5QFwzh60IIHlsSnAvps,4ckuS4Nj4FZ7i3Def3Br8W with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4jtyUzZm9WLc2AdaJ1dso7,52A8OAP8lTQKZCj4Rce92B,3850dYVgOFIXJh5U4BFEWH,27a1mYSG5tYg7dmEjWBcmL,3uKqnQ0dGBcpovf9lV2nqg,2wAJTrFhCnQyNSD3oUgTZO,2iicKPnvOKfbJZJZtuQnjy,127QTOFJsJQp5LbJbu3A1y,5Mtt6tZSZA9cXTHGSGpyh0,7JSL1X95BlGnDyXAScKolp,7im3jaJBEYivH5JKXMh1Yy,2Mpj1Ul5OFPyyP4wB62Rvi,5TCBWmEBrin7etRa4Lswr1,6Kj17Afjo1OKJYpf5VzCeo,2xnXIv68ECjDeDpaZloULO,4MggpJSpLnVQe7GP

Error while fetching features for chunk 3: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=2cYqizR4lgvp4Qu6IQ3qGN,0uFz9q2SvuKyWB6Ur1YCfI,1wHZx0LgzFHyeIZkUydNXq,6KfoDhO4XUWSbnyKjNp9c4,3eekarcy7kvN4yt5ZFzltW,6CfrYuD3YRDYdYvH9jNtXY,0utlOiJy2weVl9WTkcEWHy,5InOp6q2vvx0fShv3bzFLZ,4kdfjhj9xNkYU0R8xlDy8k,0zoGVO4bQXG8U6ChKwNgeg,3GVkPk8mqxz0itaAriG1L7,5lBzjMCr2QRoNlleZq6l7P,7uSGU4SrX7NyaqGTueSZf0,6REbwUNlppTfcnV4d4ZoZi,6gi6y1xwmVszDWkUqab1qw,43uQtNJUmlEdJtjbz6GdGL,5KpUzmHFirESqWjCrXnmWS,5yY9lUy8nbvjM1Uyo1Uqoc,12d5QFwzh60IIHlsSnAvps,4ckuS4Nj4FZ7i3Def3Br8W:
 None, reason: None
Error while fetching features for chunk 4: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=4jtyUzZm9WLc2AdaJ1dso7,52A8OAP8lTQKZCj4Rce92B,3850dYVgOFIXJh5U4BFEWH,27a1mYSG5tYg7dmEjWBcmL,3uKqnQ0dGBcpovf9lV2nqg,2wAJTrFhCnQyNSD3oUgTZO,2iicKPnvOKfbJZJZtuQnjy,127QTOFJsJQp5LbJbu3A1y,5Mtt6tZSZA9cXTHGSGpyh0,7JSL1X95BlGnDyXAScKolp,7im3jaJBEYivH5JKXMh1Yy,2Mpj1Ul5OFPyyP4wB62Rvi,5TCBW

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=22ruOqBqBRiZDiXFud4OXa,2tUL6dZf1mywCj5WvCPZw6,40iJIUlhi6renaREYGeIDS,7F9cT6hIRhnFCYP6GKS0tf,4Po2ztD6OBxolnmDagEh8o,4xJQDZxCjAJdemvWoBR9nL,6c5wQFfJApRMooKE7UQnlH,2oFdrFpXDgPcszmfWaKboI,4R67rQNSbbsR4TdUVOIdez,08F16baYbciTva9P4BvpiI,0nmW4fvqTwUDjlz13MNpJU,1dUHF4RyMmMTveJ0Rby6Xm,6jy9yJfgCsMHdu2Oz4BGKX,6WSBZroxQaO3FYEpT2ACFH,0F4FejjWi3bf5vGOkuQwPS,0UTyLmkQgPOEIz95OVAJTR,30KctD1WsHKTIYczXjip5a,6AshXllQhobwSXsdpgp41w,5W8jRrZ6tWrTrqnKRtIQBf,08LwMWf5Tcfsd752EPzFV0 with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=7EulRO07ZgchxqKUHb8Pvc,2Kh1bNimYKyUI5KQsVTQuE,4WEuEfFiczWxblHyuucSBL,4nuAslShoN77tq12fzwjUq,1NCuYqMc8hKMb4cpNTcJbD,1uHb5qa69lHLcFT38fIfUG,3cLc5HfEVkD78OA8Z6IEet,4KDNRh9Oor80z3XIxdWlui,4n6NDfYake476trCjJRNl0,3QFInJAm9eyaho5vBzxInN,3iVcZ5G6tvkXZkZKlMpIUs,3qX1dqmDdmMvvfMVfsx4Bh,6PzDwzYlw2gH6pYFQqNnFA,4Acntw4W2FfI7CDAIsCBjq,5pQZullzXsixizZmBcmXhy,2lD6AoA8qf2t4Dkf

Error while fetching features for chunk 6: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=22ruOqBqBRiZDiXFud4OXa,2tUL6dZf1mywCj5WvCPZw6,40iJIUlhi6renaREYGeIDS,7F9cT6hIRhnFCYP6GKS0tf,4Po2ztD6OBxolnmDagEh8o,4xJQDZxCjAJdemvWoBR9nL,6c5wQFfJApRMooKE7UQnlH,2oFdrFpXDgPcszmfWaKboI,4R67rQNSbbsR4TdUVOIdez,08F16baYbciTva9P4BvpiI,0nmW4fvqTwUDjlz13MNpJU,1dUHF4RyMmMTveJ0Rby6Xm,6jy9yJfgCsMHdu2Oz4BGKX,6WSBZroxQaO3FYEpT2ACFH,0F4FejjWi3bf5vGOkuQwPS,0UTyLmkQgPOEIz95OVAJTR,30KctD1WsHKTIYczXjip5a,6AshXllQhobwSXsdpgp41w,5W8jRrZ6tWrTrqnKRtIQBf,08LwMWf5Tcfsd752EPzFV0:
 None, reason: None
Error while fetching features for chunk 7: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=7EulRO07ZgchxqKUHb8Pvc,2Kh1bNimYKyUI5KQsVTQuE,4WEuEfFiczWxblHyuucSBL,4nuAslShoN77tq12fzwjUq,1NCuYqMc8hKMb4cpNTcJbD,1uHb5qa69lHLcFT38fIfUG,3cLc5HfEVkD78OA8Z6IEet,4KDNRh9Oor80z3XIxdWlui,4n6NDfYake476trCjJRNl0,3QFInJAm9eyaho5vBzxInN,3iVcZ5G6tvkXZkZKlMpIUs,3qX1dqmDdmMvvfMVfsx4Bh,6PzDw

...

Error while fetching features for chunk 12: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=3GVkPk8mqxz0itaAriG1L7,0xta6Gah3inmeHbj0e3F9a,4N9QsmvQJcGwJHAmDmBttU,0JP9xo3adEtGSdUEISiszL,1Is8hGpkGMiePASAxBluxM,0YImOCkIJ2PWhCXaURCZnY,3SAga35lAPYdjj3qyfEsCF,609mekAwxC90JkQ4BDo9IK,7wTA0NKIm6T7nP2kaymU2a,7LB3ilSpX1mSMKKztNTnu2,6gbiTbclnHlmSIPfmF2zEc,5ClRKRmW1p6RQcdwzY1kNJ,5Ee4IyhPaOVRMc15fUOeG9,4WuOWVnAqvEQxgSRrspBgt,7aRCf5cLOFN1U7kvtChY1G,14qNHvX8h6HoynFuV0RxWS,5hVghJ4KaYES3BFUATCYn0,2FqqVHvC4eKrsk97vpMRid,7EiZI6JVHllARrX9PUvAdX,4KW1lqgSr8TKrvBII0Brf8:
 None, reason: None
Error while fetching features for chunk 13: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=6tZ3b7ik1QDXFAZlXib0YZ,2e3Ea0o24lReQFR4FA7yXH,1pacwLXyRO47ka0v6LTIiY,6n849Tolbsyapue50SlLWS,1qA7crrPzO702kDZgRAhyk,4FyesJzVpA39hbYvcseO2d,0E2x4weitjqmzFilfqV4do,0lodMO0qK83vfPiiD7FMEt,7aqfrAY2p9BUSiupwk3svU,1iUlclkUb1sXcmrDqfQkIH,42VsgItocQwOQC3XWZ8JNA,0Fg88OWOdERXLMDcTWYwjf,5gF

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=4Orf7Hvci6SZrPIoUjGuFY,016z46SJwcRosW9Xb8JKmF,3eh51r6rFWAlGQRlHx9QnQ,7MmG8p0F9N3C4AXdK6o6Eb,77DRzu7ERs0TX3roZcre7Q,1wFFFzJ5EsKbBWZriAcubN,6AI3ezQ4o3HUoP6Dhudph3,6Aj2H0Z0v0JgnSfPEwCgW0,4VnYL2m6J9kWrBSw22CXwx,16utyJfWZGkTWYDobMzND7,4nva9EpKntUTs6CRSGBCn9,2Gk6fi0dqt91NKvlzGsmm7,4CdAc7V4hUe1cuJAjnPpSQ,6bLopGnirdrilrpdVB6Um1,1qIwin7JMVuX70qN6wD8ww,1D3z6HTiQsNmZxjl7F7eoG,4b7vk8SRcYgnxpk0JOIS7r,4nhcjelU9PidsBzUkazfZs,1560osUcXrnov6yuOjXvc1,5gzEL5O1ubbtIHUhLkSMTz with Params: {} returned 403 due to None
HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1lloYB6uHOTDmheiWsq4DL,69phA1R4gmQsBFRQ3INW8C,3iRCr53b2ZOvoKMTNvBn5d,6bpFKtmybxjGaO7OSOQqOc,3rXlcLZk3MCaRPND5g9QiW,0FIDCNYYjNvPVimz5icugS,6ToUq75iHEOgqD1lT0eJoh,1ITpqT1kDATCtMBwOK7sYn,0bkV1iQHSxBaksUqgEkcbc,124IHGAzY9F3unizZ08iRc,7CyPwkp0oE8Ro9Dd5CUDjW,1wGnp7ejkCuXRsehXx0PNP,1tTKy8pW1dMaXJm7bYrSzA,3TN2HZSll8ueBFtt88VbfX,4pftaoQbbheCXSdleWIeDK,0NUqi0ps17YpLUC3

Error while fetching features for chunk 15: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=4Orf7Hvci6SZrPIoUjGuFY,016z46SJwcRosW9Xb8JKmF,3eh51r6rFWAlGQRlHx9QnQ,7MmG8p0F9N3C4AXdK6o6Eb,77DRzu7ERs0TX3roZcre7Q,1wFFFzJ5EsKbBWZriAcubN,6AI3ezQ4o3HUoP6Dhudph3,6Aj2H0Z0v0JgnSfPEwCgW0,4VnYL2m6J9kWrBSw22CXwx,16utyJfWZGkTWYDobMzND7,4nva9EpKntUTs6CRSGBCn9,2Gk6fi0dqt91NKvlzGsmm7,4CdAc7V4hUe1cuJAjnPpSQ,6bLopGnirdrilrpdVB6Um1,1qIwin7JMVuX70qN6wD8ww,1D3z6HTiQsNmZxjl7F7eoG,4b7vk8SRcYgnxpk0JOIS7r,4nhcjelU9PidsBzUkazfZs,1560osUcXrnov6yuOjXvc1,5gzEL5O1ubbtIHUhLkSMTz:
 None, reason: None
Error while fetching features for chunk 16: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=1lloYB6uHOTDmheiWsq4DL,69phA1R4gmQsBFRQ3INW8C,3iRCr53b2ZOvoKMTNvBn5d,6bpFKtmybxjGaO7OSOQqOc,3rXlcLZk3MCaRPND5g9QiW,0FIDCNYYjNvPVimz5icugS,6ToUq75iHEOgqD1lT0eJoh,1ITpqT1kDATCtMBwOK7sYn,0bkV1iQHSxBaksUqgEkcbc,124IHGAzY9F3unizZ08iRc,7CyPwkp0oE8Ro9Dd5CUDjW,1wGnp7ejkCuXRsehXx0PNP,1tT

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=53WWD2BBC6Jcip8wy5CniE,42pKWuJst0oAuUbTvkNqr6,5a1ChasrKutw0MUpmGWo2w,6hPSal3ragmfUt1I5mKZjn,1OFQT3lHGbnn3x9jPtagXk,4XcZp2xqbiD8YsnPboNUDo,4Qay2jEWJYjXmJ8eNOGtaD,6VObnIkLVruX4UVyxWhlqm,7aVjnFsARHuTBZzkR6rYOR,37Qn9foJ2E2S24guZSyvyg,4AmISPiQJMPLN4KootjCUu with Params: {} returned 403 due to None


Error while fetching features for chunk 18: http status: 403, code: -1 - https://api.spotify.com/v1/audio-features/?ids=53WWD2BBC6Jcip8wy5CniE,42pKWuJst0oAuUbTvkNqr6,5a1ChasrKutw0MUpmGWo2w,6hPSal3ragmfUt1I5mKZjn,1OFQT3lHGbnn3x9jPtagXk,4XcZp2xqbiD8YsnPboNUDo,4Qay2jEWJYjXmJ8eNOGtaD,6VObnIkLVruX4UVyxWhlqm,7aVjnFsARHuTBZzkR6rYOR,37Qn9foJ2E2S24guZSyvyg,4AmISPiQJMPLN4KootjCUu:
 None, reason: None


In [19]:
audio_features

[]